IMDB sentiment analysis with word embedding

#### Step 1: Load and Preprocess the data

In [1]:
#Load the IMDB dataset
from datasets import load_dataset

#Load the imdb dataset
imdb_dataset = load_dataset("imdb")

# Take a small subset for training and testing to keep things manageable
train_data = imdb_dataset['train'].shuffle(seed=42).select(range(1000))
test_data = imdb_dataset['test'].shuffle(seed=42).select(range(500))

In [2]:
# import necessary libraries for NLP processing
import nltk
from nltk.tokenize import TreebankWordTokenizer #for breaking text into words
from nltk.stem import PorterStemmer, WordNetLemmatizer #for stemming and lemmatization
from nltk.corpus import wordnet, stopwords #for accessing word meaning and stopwords
from nltk import pos_tag #for part of speech tagging

#download required resources for tokenization, lemmatization, and stop words removal
nltk.data.path.append('/Users/smirghor/nltk_data')
nltk.download('wordnet') 
nltk.download('averaged_perceptron_tagger_eng') #POS tagger for lemmatization
nltk.download('stopwords') #stop words list

#innitialize the stemmer and lemmatizer
tokenizer = TreebankWordTokenizer()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) #define a set of common english stop words


# Function to convert POS tags to a format suitable for WordNet lemmatization
def get_wordnet_pos(tag):
    # Map POS tag to WordNet format for accurate lemmatization
    if tag.startswith('J'):
        return wordnet.ADJ  # Adjective
    elif tag.startswith('V'):
        return wordnet.VERB  # Verb
    elif tag.startswith('N'):
        return wordnet.NOUN  # Noun
    elif tag.startswith('R'):
        return wordnet.ADV  # Adverb
    else:
        return wordnet.NOUN  # Default to noun if unknown


# Function to preprocess a given text 
def preprocess_text(text):
    # 1. tokenize the text into words
    tokens = tokenizer.tokenize(text)
    # 2. apply stemming
    tokens = [stemmer.stem(token) for token in tokens if token.isalpha()]
    #3. apply lemmatization
    tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos_tag([token])[0][1])) for token in tokens]
    #4. remove stop words
    tokens = [token for token in tokens if token.lower() not in stop_words]
    # join the token back into a single string and return it
    return ' '.join(tokens)



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/smirghor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/smirghor/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/smirghor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Apply preprocessing to the training data
Preprocessed_train_data= [preprocess_text(review['text']) for review in train_data]
preprocessed_test_data = [preprocess_text(review['text']) for review in test_data]

#Extract the laels (0=Negative, 1=Positive)
y_train = [review['label'] for review in train_data]
y_test = [review['label'] for review in test_data]

#### Step 2: Convert preprocessed reviews to Vectors uning GloVe

In [4]:
#	1.	Load GloVe Embeddings:
import numpy as np 

#Define a function to load Glove embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {} # Dictionary to store the word vectors
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split() # Split the line into word and coefficients
            word = values[0] # The word is the first value
            vector = np.asarray(values[1:], dtype='float32') # Coefficients are the rest of the values
            embeddings_index[word] = vector # Store the word and its vector in the dictionary
    return embeddings_index

#Load Glove embeddings (adjust the path as needed)
glove_file_path = '/Users/smirghor/Library/Mobile Documents/com~apple~CloudDocs/Personal/Machine Learning Projects/glove.6B/glove.6B.100d.txt'

# Load the embeddings into the 'embeddings_index' variable
embeddings_index = load_glove_embeddings(glove_file_path)

# Print the number of words loaded
print(f"Loaded {len(embeddings_index)} word vectors.")

Loaded 400000 word vectors.


In [5]:
# 2. Convert Reviews to Vectors:
# function to convert a review into an average vector using GloVe
def review_to_vector(review, embedding, dim=300):
    words = review.split() #split reviews into individual words
    word_vectors = [embedding[word] for word in words if word in embedding]
    if len(word_vectors) ==0:
        return np.zeros(dim) #Return a zero vector if no words have embeddings
    return np.mean(word_vectors, axis=0) #Average the vectors

#Apply this function to training and test data
X_train_vectors = np.array([review_to_vector(review, embeddings_index) for review in Preprocessed_train_data])
X_test_vectors = np.array([review_to_vector(review, embeddings_index) for review in preprocessed_test_data])

#Check the shape of the training vector
print(f"Training Data Vector Shape: {X_train_vectors.shape}")

Training Data Vector Shape: (1000, 100)


#### Step 3: Build and train a neural network model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Define a simple neural network model 
model = Sequential([
    Dense(64, activation='relu', input_shape=(300,)), #Input layer with 64 neuron, and ReLu activation
    Dense(32, activation='relu'), #Hidden layer with 32 neuron
    Sende(1, activation='sigmoid') #Output layer with sigmoid for binary classification
])

#Compile the model with binary cross-entropy loss and th Adam optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Train the model using the training vectors
history = model.fit(X_train_vectors, np.array(y_train), epoch=10, batch_size=32, validation_split=0.2)